In [1]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained("bert-base-cased")
example="My name is Sylvain and I work at Hugging Face in Brooklyn."
encoding=tokenizer(example)
print(type(encoding))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [2]:
#to check if our tokenizer is fast or not
tokenizer.is_fast

True

In [3]:
encoding.tokens()

['[CLS]',
 'My',
 'name',
 'is',
 'S',
 '##yl',
 '##va',
 '##in',
 'and',
 'I',
 'work',
 'at',
 'Hu',
 '##gging',
 'Face',
 'in',
 'Brooklyn',
 '.',
 '[SEP]']

In [4]:
encoding.word_ids()

[None, 0, 1, 2, 3, 3, 3, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 12, None]

In [5]:
start,end=encoding.word_to_chars(3)
example[start:end]

'Sylvain'

In [6]:
#Inside token classififcation pipeline
from transformers import pipeline
token_classifier=pipeline("token-classification")
token_classifier("My name is Davitya and I work at Hugging Face in Brooklyn.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

[{'entity': 'I-PER',
  'score': 0.99701893,
  'index': 4,
  'word': 'Da',
  'start': 11,
  'end': 13},
 {'entity': 'I-PER',
  'score': 0.9848393,
  'index': 5,
  'word': '##vity',
  'start': 13,
  'end': 17},
 {'entity': 'I-PER',
  'score': 0.99675554,
  'index': 6,
  'word': '##a',
  'start': 17,
  'end': 18},
 {'entity': 'I-ORG',
  'score': 0.96651834,
  'index': 11,
  'word': 'Hu',
  'start': 33,
  'end': 35},
 {'entity': 'I-ORG',
  'score': 0.97666055,
  'index': 12,
  'word': '##gging',
  'start': 35,
  'end': 40},
 {'entity': 'I-ORG',
  'score': 0.98775,
  'index': 13,
  'word': 'Face',
  'start': 41,
  'end': 45},
 {'entity': 'I-LOC',
  'score': 0.9933444,
  'index': 15,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

In [7]:
token_classifier=pipeline("token-classification",aggregation_strategy="simple")# The aggregation_strategy picked will change the scores computed for each grouped entity. With "simple" the score is just the mean of the scores of each token in the given entity
token_classifier("My name is Davitya and I work at Hugging Face in Brooklyn.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity_group': 'PER',
  'score': 0.9928713,
  'word': 'Davitya',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.97697634,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.9933444,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

In [8]:
#from inputs to prediction
from transformers import AutoTokenizer,TFAutoModelForTokenClassification

pre_model="dbmdz/bert-large-cased-finetuned-conll03-english"
tokenizer=AutoTokenizer.from_pretrained(pre_model)
model=TFAutoModelForTokenClassification.from_pretrained(pre_model)
inputs=tokenizer(example,return_tensors="tf")
outputs=model(inputs)

All PyTorch model weights were used when initializing TFBertForTokenClassification.

All the weights of TFBertForTokenClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


In [9]:
print(inputs["input_ids"].shape)
print(outputs.logits.shape)

(1, 19)
(1, 19, 9)


In [10]:
#in above output we can see that we have 1 sequence of 19 tokens and the model has 9 different labels

In [11]:
#to provide softmax function to normlize the logits to probabilities and to get predictions

import tensorflow as tf
probabilities=tf.math.softmax(outputs.logits,axis=-1)[0]
probabilities=probabilities.numpy().tolist()
predictions=tf.math.argmax(outputs.logits,axis=-1)[0]
predictions=predictions.numpy().tolist()
print(predictions)
print(probabilities)

[0, 0, 0, 0, 4, 4, 4, 4, 0, 0, 0, 0, 6, 6, 6, 0, 8, 0, 0]
[[0.9994322061538696, 1.6470285117975436e-05, 3.4267035516677424e-05, 1.6042313291109167e-05, 8.250691462308168e-05, 2.138227500836365e-05, 0.00015649088891223073, 1.9652095943456516e-05, 0.00022089220874477178], [0.9989631175994873, 1.851577326306142e-05, 5.240462633082643e-05, 1.253474511031527e-05, 0.00043473768164403737, 3.087438381044194e-05, 0.0003146878443658352, 2.7860780392074957e-05, 0.00014510893379338086], [0.9997084140777588, 8.308127689815592e-06, 2.874564415833447e-05, 5.6503645282646175e-06, 8.694856660440564e-05, 9.783467248780653e-06, 6.78614669595845e-05, 1.1794005331466906e-05, 7.241901039378718e-05], [0.9998350143432617, 5.645536930387607e-06, 1.3955179383629002e-05, 4.313381850806763e-06, 4.017698665848002e-05, 8.123078259814065e-06, 5.648501610266976e-05, 8.991642971523106e-06, 2.723914076341316e-05], [0.00018333389016333967, 2.5156619813060388e-05, 4.846203955821693e-05, 1.490058366471203e-05, 0.999382913

In [12]:
model.config.id2label

{0: 'O',
 1: 'B-MISC',
 2: 'I-MISC',
 3: 'B-PER',
 4: 'I-PER',
 5: 'B-ORG',
 6: 'I-ORG',
 7: 'B-LOC',
 8: 'I-LOC'}

In [13]:
results=[]
tokens=inputs.tokens()

for idx,pred in enumerate(predictions):
    label=model.config.id2label[pred]
    if label !="O":
        results.append({"entity": label,"score": probabilities[idx][pred],"word": tokens[idx]})
print(results)

[{'entity': 'I-PER', 'score': 0.9993829131126404, 'word': 'S'}, {'entity': 'I-PER', 'score': 0.998154878616333, 'word': '##yl'}, {'entity': 'I-PER', 'score': 0.995907187461853, 'word': '##va'}, {'entity': 'I-PER', 'score': 0.9992326498031616, 'word': '##in'}, {'entity': 'I-ORG', 'score': 0.9738930463790894, 'word': 'Hu'}, {'entity': 'I-ORG', 'score': 0.9761150479316711, 'word': '##gging'}, {'entity': 'I-ORG', 'score': 0.9887974858283997, 'word': 'Face'}, {'entity': 'I-LOC', 'score': 0.9932106137275696, 'word': 'Brooklyn'}]


In [14]:
inputs_with_offsets=tokenizer(example,return_offsets_mapping=True)
inputs_with_offsets["offset_mapping"]

[(0, 0),
 (0, 2),
 (3, 7),
 (8, 10),
 (11, 12),
 (12, 14),
 (14, 16),
 (16, 18),
 (19, 22),
 (23, 24),
 (25, 29),
 (30, 32),
 (33, 35),
 (35, 40),
 (41, 45),
 (46, 48),
 (49, 57),
 (57, 58),
 (0, 0)]

In [15]:
results=[]
inputs_with_offsets=tokenizer(example,return_offsets_mapping=True)
tokens=inputs_with_offsets.tokens()
offsets=inputs_with_offsets["offset_mapping"]

for idx,pred in enumerate(predictions):
    label=model.config.id2label[pred]
    if label != "O":
      start,end=offsets[idx]
      results.append({"entity": label, "score": probabilities[idx][pred], "word": tokens[idx],"start":start,"end":end})
print(results)

[{'entity': 'I-PER', 'score': 0.9993829131126404, 'word': 'S', 'start': 11, 'end': 12}, {'entity': 'I-PER', 'score': 0.998154878616333, 'word': '##yl', 'start': 12, 'end': 14}, {'entity': 'I-PER', 'score': 0.995907187461853, 'word': '##va', 'start': 14, 'end': 16}, {'entity': 'I-PER', 'score': 0.9992326498031616, 'word': '##in', 'start': 16, 'end': 18}, {'entity': 'I-ORG', 'score': 0.9738930463790894, 'word': 'Hu', 'start': 33, 'end': 35}, {'entity': 'I-ORG', 'score': 0.9761150479316711, 'word': '##gging', 'start': 35, 'end': 40}, {'entity': 'I-ORG', 'score': 0.9887974858283997, 'word': 'Face', 'start': 41, 'end': 45}, {'entity': 'I-LOC', 'score': 0.9932106137275696, 'word': 'Brooklyn', 'start': 49, 'end': 57}]


In [16]:
#Inside Question Answers pipeline
# A question-answer pipeline in natural language processing (NLP) typically refers to a system that takes a user’s question as input and returns the most relevant answer

from transformers import pipeline
question_answer_pipeline=pipeline("question-answering")
context="""Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch, and TensorFlow — with a seamless integrationbetween them. It's straightforward to train your models with one before loading them for inference with the other."""
question="Which deep learning libraries back Transformers?"
question_answer_pipeline(context=context,question=question)

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

{'score': 0.9759499430656433,
 'start': 75,
 'end': 103,
 'answer': 'Jax, PyTorch, and TensorFlow'}

In [17]:
#using a model for question-answering
from transformers import AutoTokenizer,TFAutoModelForQuestionAnswering
pre_model="distilbert-base-cased-distilled-squad"
tokenizer=AutoTokenizer.from_pretrained(pre_model)
model=TFAutoModelForQuestionAnswering.from_pretrained(pre_model)
inputs=tokenizer(question,context,return_tensors="tf")
outputs=model(inputs)
outputs

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFDistilBertForQuestionAnswering.

All the weights of TFDistilBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


TFQuestionAnsweringModelOutput(loss=None, start_logits=<tf.Tensor: shape=(1, 67), dtype=float32, numpy=
array([[-3.9840026 , -5.9988594 , -4.097372  , -6.365857  , -6.497088  ,
        -7.1876926 , -6.137077  , -5.273674  , -4.1051836 ,  1.0072219 ,
        -4.0101433 , -1.6448282 , -2.3012624 ,  3.1291273 ,  6.241598  ,
        -0.64098483, -1.6363262 , -0.6010818 , -4.3990536 , -1.6813371 ,
         2.5224493 , 10.919356  , -0.70599586,  2.9258683 , -1.2581276 ,
        -1.9066119 , -3.8923395 , -1.7415618 , -1.6038619 , -1.8832853 ,
         1.9336158 , -1.2807457 , -0.9327436 , -0.05147028, -1.3880672 ,
        -3.4544284 , -3.8845766 , -3.4083333 , -6.9841986 , -5.754724  ,
        -3.461074  , -6.0376577 , -6.9105544 , -6.631365  , -3.3872998 ,
        -4.3276405 , -2.3212216 , -6.0577564 , -7.212279  , -3.720534  ,
        -6.6120043 , -3.7556007 , -3.502877  , -2.6876569 , -7.3075404 ,
        -4.2215595 , -6.862074  , -4.422335  , -6.624553  , -7.2748446 ,
        -5.8594575 ,

In [18]:
start_logits=outputs.start_logits
end_logits=outputs.end_logits
print(start_logits.shape,end_logits.shape)

(1, 67) (1, 67)


In [19]:
#masking the indices that are not part of the context
import tensorflow as tf
sequence_ids=inputs.sequence_ids()
mask=[i!=1 for i in sequence_ids]

mask[0]=False# this ensure that [CLS] token is unmasked
mask=tf.constant(mask)[None]#  Converts the mask (which is currently a Python list) into a TensorFlow constant (tensor).[None]: This adds a new dimension to the tensor. This is useful because, typically, in a batch setting, you'd need to provide a mask for every input in the batch. Adding this extra dimension helps when handling multiple inputs (batch processing).

#  it replaces values in start_logits and end_logits with a large negative number (-100000) wherever the mask is True.
start_logits=tf.where(mask,-10000,start_logits)
end_logits=tf.where(mask,-10000,end_logits)
# By setting logits to a very negative value for masked tokens, you're effectively preventing the model from selecting those positions as the start or end of the answer.

print(mask)
print(start_logits)
print(end_logits)

tf.Tensor(
[[False  True  True  True  True  True  True  True  True False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False  True]], shape=(1, 67), dtype=bool)
tf.Tensor(
[[-3.9840026e+00 -1.0000000e+04 -1.0000000e+04 -1.0000000e+04
  -1.0000000e+04 -1.0000000e+04 -1.0000000e+04 -1.0000000e+04
  -1.0000000e+04  1.0072219e+00 -4.0101433e+00 -1.6448282e+00
  -2.3012624e+00  3.1291273e+00  6.2415981e+00 -6.4098483e-01
  -1.6363262e+00 -6.0108179e-01 -4.3990536e+00 -1.6813371e+00
   2.5224493e+00  1.0919356e+01 -7.0599586e-01  2.9258683e+00
  -1.2581276e+00 -1.9066119e+00 -3.8923395e+00 -1.7415618e+00
  -1.6038619e+00 -1.8832853e+00  1.9336158e+00 -1.2807457e+00
  -9.3274361e-01 -5.1470280e-02 -1.388

In [20]:
# applying softmax function on the logits to get probabilities
start_probabilities=tf.math.softmax(start_logits,axis=-1)[0].numpy()
end_probabilities=tf.math.softmax(end_logits,axis=-1)[0].numpy()
print(start_probabilities)
print(end_probabilities)

[3.3342221e-07 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 4.9051883e-05
 3.2481910e-07 3.4584775e-06 1.7939027e-06 4.0943734e-04 9.2027215e-03
 9.4373181e-06 3.4880070e-06 9.8215069e-06 2.2016077e-07 3.3344875e-06
 2.2320841e-04 9.8955709e-01 8.8433071e-06 3.3412810e-04 5.0912795e-06
 2.6619127e-06 3.6542900e-07 3.1395973e-06 3.6030999e-06 2.7247345e-06
 1.2387488e-04 4.9774162e-06 7.0491878e-06 1.7016537e-05 4.4708977e-06
 5.6622019e-07 3.6827680e-07 5.9293092e-07 1.6596854e-08 5.6751841e-08
 5.6247001e-07 4.2766452e-08 1.7865256e-08 2.3618913e-08 6.0553486e-07
 2.3645813e-07 1.7584540e-06 4.1915523e-08 1.3212090e-08 4.3392745e-07
 2.4080620e-08 4.1897479e-07 5.3944154e-07 1.2189612e-06 1.2011586e-08
 2.6292062e-07 1.8752718e-08 2.1509419e-07 2.3780334e-08 1.2410806e-08
 5.1108646e-08 2.1943853e-08 5.2277631e-09 2.3414021e-08 3.8696374e-08
 3.8908563e-08 0.0000000e+00]
[1.82573706e-06 0.00000000e+00 0.00000000e+00 0

In [21]:
scores=start_probabilities[:,None]*end_probabilities[None,:]

In [22]:
import numpy as np
scores=np.triu(scores)# This function extracts the upper triangular part of a matrix, meaning only the values above and on the diagonal are kept. All values below the diagonal are set to zero.

max_index=scores.argmax().item()
start_index=max_index// scores.shape[1]
end_index=max_index % scores.shape[1]
score=scores[start_index,end_index]

inputs_with_offsets=tokenizer(question,context,return_offsets_mapping=True)
offsets=inputs_with_offsets["offset_mapping"]

start_char, _ =offsets[start_index]
_,end_char=offsets[end_index]
answer=context[start_char:end_char]

print(f"The answer: '{answer}',score: {score:.4f}")

The answer: 'Jax, PyTorch, and TensorFlow',score: 0.9759


In [23]:
result = {
    "answer": answer,
    "start": start_char,
    "end": end_char,
    "score": scores[start_index, end_index],
}
print(result)

{'answer': 'Jax, PyTorch, and TensorFlow', 'start': 75, 'end': 103, 'score': 0.9759497}


In [24]:
#handling long contexts
long_context = """
🤗 Transformers: State of the Art NLP

🤗 Transformers provides thousands of pretrained models to perform tasks on texts such as classification, information extraction,
question answering, summarization, translation, text generation and more in over 100 languages.
Its aim is to make cutting-edge NLP easier to use for everyone.

🤗 Transformers provides APIs to quickly download and use those pretrained models on a given text, fine-tune them on your own datasets and
then share them with the community on our model hub. At the same time, each python module defining an architecture is fully standalone and
can be modified to enable quick research experiments.

Why should I use transformers?

1. Easy-to-use state-of-the-art models:
  - High performance on NLU and NLG tasks.
  - Low barrier to entry for educators and practitioners.
  - Few user-facing abstractions with just three classes to learn.
  - A unified API for using all our pretrained models.
  - Lower compute costs, smaller carbon footprint:

2. Researchers can share trained models instead of always retraining.
  - Practitioners can reduce compute time and production costs.
  - Dozens of architectures with over 10,000 pretrained models, some in more than 100 languages.

3. Choose the right framework for every part of a model's lifetime:
  - Train state-of-the-art models in 3 lines of code.
  - Move a single model between TF2.0/PyTorch frameworks at will.
  - Seamlessly pick the right framework for training, evaluation and production.

4. Easily customize a model or an example to your needs:
  - We provide examples for each architecture to reproduce the results published by its original authors.
  - Model internals are exposed as consistently as possible.
  - Model files can be used independently of the library for quick experiments.

🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch and TensorFlow — with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""

inputs=tokenizer(question,long_context)
print(len(inputs["input_ids"]))

460


In [25]:
#its really long so we need to do truncation
inputs=tokenizer(question,long_context,max_length=384,truncation="only_second")
print(tokenizer.decode(inputs["input_ids"]))

[CLS] Which deep learning libraries back Transformers? [SEP] [UNK] Transformers : State of the Art NLP [UNK] Transformers provides thousands of pretrained models to perform tasks on texts such as classification, information extraction, question answering, summarization, translation, text generation and more in over 100 languages. Its aim is to make cutting - edge NLP easier to use for everyone. [UNK] Transformers provides APIs to quickly download and use those pretrained models on a given text, fine - tune them on your own datasets and then share them with the community on our model hub. At the same time, each python module defining an architecture is fully standalone and can be modified to enable quick research experiments. Why should I use transformers? 1. Easy - to - use state - of - the - art models : - High performance on NLU and NLG tasks. - Low barrier to entry for educators and practitioners. - Few user - facing abstractions with just three classes to learn. - A unified API for

In [26]:
#if we see in the above output the answe was in the end but now it was truncated so in removed from the context which is wrong
#this means the model will have hard tim epicking up the correct answer. To fix this, we split the context into smaller parts

In [27]:
#for example
sentence="This sentence is not too long but we are going to split it anyway."
inputs1=tokenizer(sentence,truncation=True,return_overflowing_tokens=True,max_length=6,stride=2)
for ids in inputs1["input_ids"]:
  print(tokenizer.decode(ids))

[CLS] This sentence is not [SEP]
[CLS] is not too long [SEP]
[CLS] too long but we [SEP]
[CLS] but we are going [SEP]
[CLS] are going to split [SEP]
[CLS] to split it anyway [SEP]
[CLS] it anyway. [SEP]


In [28]:
#every chunk has 6 tokens and there is an overlap of two tokens
#we need to add padding in the last sentence as well to make all the sentences of equal length

In [29]:
print(inputs1.keys())

dict_keys(['input_ids', 'attention_mask', 'overflow_to_sample_mapping'])


In [30]:
#The last key, overflow_to_sample_mapping, is a map that tells us which sentence each of the results corresponds to

In [31]:
print(inputs1["overflow_to_sample_mapping"])

[0, 0, 0, 0, 0, 0, 0]


In [32]:
sentences=["This sentence is not too long but we are going to split it anyway.","This sentence is shorter but will still get split.",]
inputs=tokenizer(sentences,truncation=True,return_overflowing_tokens=True,max_length=6,stride=2)
print(inputs["overflow_to_sample_mapping"])

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]


In [33]:
inputs=tokenizer(
    question,
    long_context,
    stride=128,
    max_length=384,
    padding="longest",
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

In [34]:
_=inputs.pop("overflow_to_sample_mapping")
offsets=inputs.pop("offset_mapping")
inputs=inputs.convert_to_tensors("tf")
print(inputs["input_ids"].shape)

(2, 384)


In [35]:
outputs=model(inputs)
start_logits=outputs.start_logits
end_logits=outputs.end_logits
print(start_logits.shape,end_logits.shape)

(2, 384) (2, 384)


In [36]:
sequence_ids=inputs.sequence_ids()
# Mask everything apart from the tokens of the context
mask=[i!=1 for i in sequence_ids]
# Unmask the [CLS] token
mask[0]=False
# Mask all the [PAD] tokens
mask=tf.math.logical_or(tf.constant(mask)[None],inputs["attention_mask"]==0)
start_logits=tf.where(mask,-10000,start_logits)
end_logits=tf.where(mask,-10000,end_logits)

In [37]:
start_probabilities=tf.math.softmax(start_logits,axis=-1).numpy()
end_probabilities=tf.math.softmax(end_logits,axis=-1).numpy()

In [38]:
candidates = []
for start_probs, end_probs in zip(start_probabilities, end_probabilities):
    scores = start_probs[:, None] * end_probs[None, :]
    idx = np.triu(scores).argmax().item()

    start_idx = idx // scores.shape[1]
    end_idx = idx % scores.shape[1]
    score = scores[start_idx, end_idx].item()
    candidates.append((start_idx, end_idx, score))

print(candidates)

[(0, 0, 0.8486798405647278), (171, 182, 0.9766979813575745)]


In [39]:
for candidate, offset in zip(candidates, offsets):
    start_token, end_token, score = candidate
    start_char, _ = offset[start_token]
    _, end_char = offset[end_token]
    answer = long_context[start_char:end_char]
    result = {"answer": answer, "start": start_char, "end": end_char, "score": score}
    print(result)

{'answer': '', 'start': 0, 'end': 0, 'score': 0.8486798405647278}
{'answer': 'Jax, PyTorch and TensorFlow', 'start': 1892, 'end': 1919, 'score': 0.9766979813575745}
